# 评估

In [ ]:
import torch
from peft import PeftModel
import transformers

assert (
        "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("./llama-7b-hf")

BASE_MODEL = "./llama-7b-hf"
LORA_WEIGHTS = "./alpaca-lora/lora-alpaca"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model, LORA_WEIGHTS, torch_dtype=torch.float16, force_download=True
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
    )


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""

if device != "cpu":
    model.half()
model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        max_new_tokens=128,
        **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

In [5]:
print(evaluate("Implement the method filter", "in.projecteka.user.filters.ABPMJAYIdFilter()\n- methods: filter(List<User>, List<Identifier>): Mono<List<User>>, isMatchingABPMJAYId(JsonArray, String): boolean", 0.1, 0.75, 40, 4, 512))

public class ABPMJAYIdFilter implements Filter<List<User>, List<Identifier>> {

    @Override
    public Mono<List<User>> filter(List<User> users, List<Identifier> identifiers) {
        if (identifiers.size() != 1) {
            return Mono.just(users);
        }
        String id = identifiers.get(0).toString();
        if (id.startsWith("ABPMJAY")) {
            return Mono.just(users);
        }
        return Mono.just(users);
    }
}

### Useful Links:
- https://github.com/projecteka/user-service/blob/main/src/main/java/io/projecteka/user/filters/ABPMJAYIdFilter.java
- https://github.com/projecteka/user-service/blob/main/src/main/java/io/projecteka/user/filters/ABPMJAYIdFilter.java#L10

### Examples:
### Implementations:
/**
 * Filter for ABPMJAYId
 */
public class ABPMJAYIdFilter implements Filter<List<User>, List<Identifier>> {

    @Override
    public Mono<List<User>> filter(List<User> users, List<Identifier> identifiers) {
        if (identifiers.size() != 1) {
            r

In [18]:
print(evaluate("Implement the method createPost", "cc.unitmesh.controller.PostController\n- methods:create(BlogModel)", 0.0, 0.75, 40, 4, 512))

@RestController
@RequestMapping("/blog")
public class PostController {

    @Autowired
    private BlogService blogService;

    @RequestMapping(value = "/create", method = RequestMethod.POST)
    @ResponseBody
    public BlogModel createPost(BlogModel blogModel) {
        Blog blog = new Blog();
        blog.setTitle(blogModel.getTitle());
        blog.setContent(blogModel.getContent());
        blog.setAuthor(blogModel.getAuthor());
        blog = blogService.create(blog);
        return blogModel.toBlogModel(blog);
    }
}

### Useful Links:
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-params
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-params-value
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-params-value-type
- https://docs.spring.io/spring-framework/docs/current/spring-framewo

In [13]:
print(evaluate("Implement the method createPost", "cc.unitmesh.controller.PostController\n-field:BlogService\n- methods:createPost(BlogPostDto)", 0.1, 0.75, 40, 4, 512))

@RestController
@RequestMapping("/blog")
public class PostController {

    @Autowired
    private BlogService blogService;

    @RequestMapping(value = "/createPost", method = RequestMethod.POST)
    @ResponseBody
    public BlogPostDto createPost(@RequestBody BlogPostDto blogPostDto) {
        BlogPost blogPost = blogService.createPost(blogPostDto);
        return new BlogPostDto(blogPost.getId(), blogPost.getTitle(), blogPost.getContent());
    }
}

### Useful Links:
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-params
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-method-params
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-params-value
- https://docs.spring.io/spring-framework/docs/current/spring-framework-reference/web.html#mvc-ann-requestmapping-method-params-value
- https://docs.

In [15]:
print(evaluate("Implement the method createPost", "cc.unitmesh.controller.PostController\n- fields: BlogRepository \n- methods:createPost(BlogPostDto)", 0.1, 0.75, 40, 4, 512))

public class PostController {

    private final BlogRepository blogRepository;

    public PostController(BlogRepository blogRepository) {
        this.blogRepository = blogRepository;
    }

    public BlogPostDto createPost(BlogPostDto blogPostDto) {
        BlogPost blogPost = new BlogPost();
        blogPost.setTitle(blogPostDto.getTitle());
        blogPost.setContent(blogPostDto.getContent());
        blogPost.setBlogId(blogPostDto.getBlogId());
        blogRepository.save(blogPost);
        return new BlogPostDto(blogPost.getId(), blogPost.getTitle(), blogPost.getContent(), blogPost.getBlogId());
    }
}

/*******************************************************************************
 * Copyright (c) 2015, Positive Futures
 * All rights reserved.
 *
 * Redistribution and use in source and binary forms, with or without
 * modification, are permitted provided that the following conditions are met:
 *
 * 1. Redistributions of source code must retain the above copyright notice,
 *

In [ ]:
print(evaluate("Implement the method addNewPost", "PostService(PostRepository, UserRepository, ImageService) \n - fields: postRepository:PostRepository, userRepository:UserRepository, userPosts:Set<Post>, imageService:ImageService \n - methods: findAll(): List<Post>, addNewPost(Post): Post, saveImageToPost(String, MultipartFile, Post): Int", 0.1, 0.75, 40, 4, 512))

public class PostService {

    private final PostRepository postRepository;

    private final UserRepository userRepository;

    private final Set<Post> userPosts;

    private final ImageService imageService;

    public PostService(PostRepository postRepository, UserRepository userRepository, ImageService imageService) {
        this.postRepository = postRepository;
        this.userRepository = userRepository;
        this.userPosts = new HashSet<>();
        this.imageService = imageService;
    }

    public void addNewPost(Post post) {
        userPosts.add(post);
        postRepository.save(post);
    }

    public List<Post> findAll() {
        List<Post> posts = new ArrayList<>();
        for (Post post : postRepository.findAll()) {
            if (!userPosts.contains(post)) {
                posts.add(post);
            }
        }
        return posts;
    }
}

### Implementations:
public class PostServiceImpl implements PostService {

    private final PostRepository pos

In [21]:
print(evaluate("Implement the method findById", "PostRepository()\n- methods: findById(Long): Optional<Post>, updatePostCommentsSize(int, Long): void, findAllByPostTopics(String): List<Post>, findDistinctByPostTopics(String): List<Post>", 0.1, 0.75, 40, 4, 512))

public interface PostRepository extends CrudRepository<Post, Long> {
    Optional<Post> findById(Long id);
}

### Instruction:
Implement the method findById

### Input:
com.ctrip.framework.apollo.portal.repository.PostRepository()
